<a href="https://colab.research.google.com/github/ildimas/NeuralNetworksInManagment/blob/main/%D0%98%D0%BB%D1%8C%D1%8E%D1%89%D0%B5%D0%BD%D1%8F_%D0%9D%D0%A2%D0%92%D0%A3_1_%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1: Нейронные сети в классификации изображений

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

## 1. Подготовка данных

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

full_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

train_size = 50000
val_size = 10000
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

batch_sizes = [32, 64, 128]
train_loaders = {}
val_loaders = {}
test_loaders = {}

for batch_size in batch_sizes:
    train_loaders[batch_size] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loaders[batch_size] = DataLoader(val_dataset, batch_size=batch_size)
    test_loaders[batch_size] = DataLoader(test_dataset, batch_size=batch_size)

## 2. Имплементация моделей

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.layers(x)

class MediumMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.layers(x)

class DeepMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.layers(x)

In [ ]:
class BasicCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7*7*32, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

In [ ]:
class ComplexCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7*7*64, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

## 3. Тренировочная функция

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100. * correct / total

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

    return train_losses, val_losses, train_accs, val_accs

In [ ]:
def plot_metrics(train_losses, val_losses, train_accs, val_accs, title):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(train_losses, label='Train Loss')
    ax1.plot(val_losses, label='Val Loss')
    ax1.set_title(f'{title} - Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()

    ax2.plot(train_accs, label='Train Accuracy')
    ax2.plot(val_accs, label='Val Accuracy')
    ax2.set_title(f'{title} - Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()

    plt.tight_layout()
    plt.show()

## 4. Обучение и оценка

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

models = {
    'Simple MLP': SimpleMLP(),
    'Medium MLP': MediumMLP(),
    'Deep MLP': DeepMLP(),
    'Basic CNN': BasicCNN(),
    'Complex CNN': ComplexCNN()
}

for name, model in models.items():
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f'\nTraining {name}...')
    train_losses, val_losses, train_accs, val_accs = train_model(
        model, train_loaders[64], val_loaders[64], criterion, optimizer, num_epochs=20, device=device
    )

    plot_metrics(train_losses, val_losses, train_accs, val_accs, name)

## 5. CIFAR-10

In [ ]:
class CIFAR10CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(8*8*64, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

In [ ]:
cifar_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=cifar_transform)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=cifar_transform)

cifar_train_loader = DataLoader(cifar_train, batch_size=64, shuffle=True)
cifar_test_loader = DataLoader(cifar_test, batch_size=64)

cifar_model = CIFAR10CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cifar_model.parameters(), lr=0.001)

print('Training CIFAR-10 model...')
train_losses, val_losses, train_accs, val_accs = train_model(
    cifar_model, cifar_train_loader, cifar_test_loader, criterion, optimizer, num_epochs=10, device=device
)

plot_metrics(train_losses, val_losses, train_accs, val_accs, 'CIFAR-10 CNN')

#### Почему важна нормализация данных и как она влияет на процесс обучения?
Нормализация приводит все входные признаки к сопоставимому масштабу (например, [0,1] или со средним 0 и дисперсией 1). Это помогает:
- Ускорить сходимость градиентного спуска
- Избежать доминирования признаков с большими значениями
- Повысить устойчивость модели

#### Как изменение batch_size влияет на скорость обучения и сходимость?
- `batch_size=32`: стабильное обновление градиента, медленное обучение, лучшее обобщение
- `batch_size=64`: баланс между шумом и скоростью
- `batch_size=128`: быстрее обучение, но выше риск переобучения или плохого минимума

> Эксперименты показывают, что `batch_size=64` часто является оптимумом между скоростью и качеством.

#### Почему важно разделять данные на тренировочную, валидационную и тестовую выборки?
- **Тренировочная**: обучение модели
- **Валидационная**: подбор гиперпараметров и наблюдение переобучения
- **Тестовая**: финальная оценка на "невиданных" данных
Без валидации невозможно контролировать переобучение и сравнивать модели объективно.


#### Как количество параметров влияет на способность к обучению?
- Меньше параметров: модель может недообучиться (underfitting)
- Больше параметров: выше гибкость, но и выше риск переобучения
Нужно находить баланс через валидацию.

#### Сравнение функций активации:
- `ReLU`: быстрая сходимость, популярна, но страдает от "затухающего градиента" для отрицательных значений
- `Sigmoid`: плохо масштабируется, страдает от vanishing gradient
- `Tanh`: центрирована, но тоже страдает от затухающего градиента
- `LeakyReLU`: решает проблему ReLU с нулевыми градиентами

> На практике ReLU/LeakyReLU дают лучшие результаты по скорости и качеству.

#### Что происходит с градиентами при увеличении глубины сети?
- Возникает **затухание** или **взрыв градиентов**
- Это замедляет или делает невозможным обучение
- Решения: нормализация (BatchNorm), инициализация, архитектуры с остаточными связями (ResNet)


#### Роль Padding и MaxPooling:
- **Padding**: сохраняет размер выходной карты признаков, предотвращает потерю информации по краям
- **MaxPooling**: уменьшает размерность, делает модель устойчивее к смещениям

> Уменьшение размерности снижает количество параметров и ускоряет обучение

#### Заменить MaxPooling на AveragePooling — что изменится?
- Поведение станет более "плавным"
- Потеряется акцент на ярко выраженных признаках
- Accuracy может снизиться
> Рекомендуется провести эксперимент и сравнить на валидации.

#### Почему CNN эффективнее MLP для изображений?
- CNN использует **локальные связи** и **shared weights**
- Позволяет эффективно извлекать **локальные признаки**
- MLP не учитывает структуру изображения, требует больше параметров


#### Как влияет последовательное применение 2 Conv-слоев без Pooling?
- Увеличивается **рецептивное поле**
- Модель видит более глобальные шаблоны, но без снижения размерности

#### Сравнение производительности базовой и усложненной CNN:
- Усложнённая CNN (больше слоёв/фильтров): выше качество на сложных задачах
- Но: дольше обучение и выше риск переобучения
> Баланс достигается через регуляризацию и валидацию

#### Влияние размера kernel_size (эксперимент):
- `3x3`: захватывает мелкие детали
- `5x5`: компромисс между локальным и глобальным
- `7x7`: крупные шаблоны, но больше параметров и риск переобучения

> Эксперименты показывают, что два слоя с `3x3` ядрами часто работают лучше, чем один `5x5` или `7x7`.


#### Как определить переобучение?
- Точность на валидации начинает ухудшаться, а на тренировке — расти
- Увеличение loss на валидации при уменьшении loss на обучении

#### Сравнение оптимизаторов:
- **SGD**: прост, но медленно сходится
- **RMSprop**: ускоряет за счёт адаптивных шагов
- **Adam**: сочетает преимущества Momentum и RMSprop, хорошее качество и скорость

> Adam чаще всего работает лучше "из коробки"

#### Влияние learning rate (эксперимент):
- `0.0001`: медленное обучение, стабильное
- `0.001`: обычно оптимальное
- `0.01`: быстрое, но может не сойтись (перескакивает минимум)

#### Сравнение early stopping и L2-регуляризации:
- **Early stopping**: прерывает обучение, если метрика валидации не улучшается
- **L2-регуляризация**: штрафует большие веса

> Совместное использование даёт наилучший эффект


#### Какие архитектурные изменения требуются для CIFAR-10?
- Увеличение глубины сети (из-за большего разнообразия классов)
- BatchNorm, Dropout, Residual connections — для устойчивости

#### Сравнение FashionMNIST и CIFAR-10:
- FashionMNIST: grayscale, простые формы
- CIFAR-10: цветные изображения, более сложные паттерны
> Модели на CIFAR-10 требуют больше параметров и слоёв

#### Transfer learning:
- Использование предварительно обученных моделей (например, ResNet на ImageNet)
- Замена классификатора на новую задачу
> Ускоряет обучение и повышает точность

#### Аугментации для CIFAR-10:
- **Horizontal flip**, **Random crop**, **Color jitter**, **Rotation**
> Улучшают обобщающую способность, имитируют разнообразие входов
